In [1]:
import pandas as pd
import sqlite3

In [2]:
orders_df = pd.read_csv("orders.csv")
print(orders_df.head())


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [3]:
users_df = pd.read_json("users.json")
print(users_df.head())


   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [4]:
# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Execute SQL file
with open("restaurants.sql", "r") as file:
    cursor.executescript(file.read())

# Read restaurants table into DataFrame
restaurants_df = pd.read_sql_query(
    "SELECT * FROM restaurants",
    conn
)

print(restaurants_df.head())


   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [5]:
orders_users_df = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)


In [6]:
final_df = pd.merge(
    orders_users_df,
    restaurants_df,
    on="restaurant_id",
    how="left"
)


In [7]:
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)


In [8]:
final_df["order_date"] = pd.to_datetime(
    final_df["order_date"],
    dayfirst=True
)


In [9]:
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(gold_city_revenue)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [10]:
avg_order_by_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(avg_order_by_cuisine)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [11]:
user_total_spend = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

high_value_users = user_total_spend[user_total_spend > 1000]

print("Number of users:", high_value_users.count())


Number of users: 2544


In [12]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

rating_revenue = (
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(rating_revenue)


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64


C:\Users\Admin\AppData\Local\Temp\ipykernel_18392\1799484758.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


In [13]:
gold_city_avg = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(gold_city_avg)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [14]:
restaurant_count = (
    final_df
    .groupby("cuisine")["restaurant_name"]
    .nunique()
)

revenue_by_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .sum()
)

analysis_df = pd.concat(
    [restaurant_count, revenue_by_cuisine],
    axis=1
)
analysis_df.columns = ["restaurant_count", "total_revenue"]

print(analysis_df.sort_values(
    by=["restaurant_count", "total_revenue"],
    ascending=[True, False]
))


KeyError: 'Column not found: restaurant_name'

In [15]:
restaurant_stats = (
    final_df
    .groupby("restaurant_name")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

filtered_restaurants = restaurant_stats[
    restaurant_stats["total_orders"] < 20
].sort_values(
    by="avg_order_value",
    ascending=False
)

print(filtered_restaurants.head())


KeyError: 'restaurant_name'

In [16]:
combo_revenue = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(combo_revenue)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64


In [17]:
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

quarter_revenue = (
    final_df
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(quarter_revenue)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64


In [18]:
import pandas as pd

# Load the final dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Count total orders placed by Gold members
gold_orders_count = df[df["membership"] == "Gold"].shape[0]

print(gold_orders_count)


4987


In [19]:
hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)

print(hyderabad_revenue)

1889367


In [20]:
distinct_users = df["user_id"].nunique()

print(distinct_users)

2883


In [21]:
avg_gold_order_value = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
    2
)

print(avg_gold_order_value)

797.15


In [22]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]

print(high_rating_orders)

3374


In [23]:
gold_df = df[df["membership"] == "Gold"]

# Find the top revenue city among Gold members
top_city = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

# Count orders placed in that city by Gold members
orders_top_city = gold_df[gold_df["city"] == top_city].shape[0]

print(top_city, orders_top_city)

Chennai 1337
